# Spaceship Titanic Pipeline 
---

## Imports and Data


In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import random



In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

## Pipelines

## Transformers

Plan: 

numeric_columns:
all numeric columns are going to be imputed with mean

cat_columns:
all cat columns are going to be imputed with mode

In [3]:
y = pd.DataFrame(train['Transported'])
X = pd.DataFrame(train.drop(columns='Transported'))

In [4]:
numeric_cols = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
cat_cols = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']

In [5]:
numeric_transform = SimpleImputer(strategy='mean')
cat_transform = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

In [6]:
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transform, numeric_cols),
    ('cat', cat_transform, cat_cols)
])

In [7]:
model = XGBClassifier(random_state=42)

In [8]:
my_pipeline = Pipeline(steps=[
    ('prepreocessor', preprocessor),
    ('model', model)
])

In [9]:
xgb_params = {
    'model__max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
    'model__learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'model__n_estimators': [50, 100, 200, 300, 400, 500],
    'model__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'model__gamma': [0, 0.1, 0.2, 0.3, 0.4, 0.5]
}

In [ ]:
random_search = RandomizedSearchCV(my_pipeline, param_distributions=xgb_params, n_iter=10, cv=5, scoring='accuracy')
random_search.fit(X, y)

In [67]:
final_model = random_search.best_estimator_

my_pipeline.set_params(model=final_model)

Pipeline(steps=[('prepreocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['Age', 'RoomService',
                                                   'FoodCourt', 'ShoppingMall',
                                                   'Spa', 'VRDeck']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['HomePlanet', 'CryoSleep',
                                                   'Cabin', 'Destination',
                                                   'VIP'])])),
                ('model',
                 Pi...
                                                grow_policy=None,
                                                importance_type=None,
                                                interaction_constraints=None,
                                                learning_rate=0.01,
                                                max_bin=None,
                                                max_cat_threshold=None,
                                                max_cat_to_onehot=None,
                                                max_delta_step=None,
                                                max_depth=5, max_leaves=None,
                                                min_child_weight=None,
                                                missing=nan,
                                                monotone_constraints=None,
                                                multi_strategy=None,
                                                n_estimators=500, n_jobs=None,
                                                num_parallel_tree=None,
                                                random_state=42, ...))]))])

In [68]:

scores = cross_val_score(my_pipeline, X, y, cv=5)

print("MAE scores:\n", scores)
print('Mean Score: ', scores.mean())

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/_indexing.py", line 338, in _get_column_indices
    all_columns = X.columns
                  ^^^^^^^^^
AttributeError: 'csr_matrix' object has no attribute 'columns'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/codespace/.local/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(es